# AVANCE 2
## Análisis de Ventas 

Este notebook demuestra el uso de los principales componentes del sistema:
- Conexión a la base de datos con SQLAlchemy
- Resultados de queries convertidos a pandas DataFrame
- Patrón Singleton aplicado a la clase de conexión
- Ejecución de pruebas unitarias con `pytest`

### Configurar la importacion correcta de modulos

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

### Conexion a la base de datos

In [2]:
from src.database.connection import DatabaseConnection
db = DatabaseConnection()
session = db.get_session()
db.close_session(session)
print("Conexión y sesión establecida correctamente.")

2025-06-08 06:15:10 - INFO - src.database.connection - Obteniendo sesión de la base de datos...
2025-06-08 06:15:10 - INFO - src.database.connection - Cerrando sesión de la base de datos...


Conexión y sesión establecida correctamente.


### Ejecucion de queries

In [3]:
# Ejecución de query y conversión a DataFrame
df = db.execute_query_as_dataframe("SELECT * FROM categories")
df.head()


2025-06-08 06:15:10 - INFO - src.database.connection - Ejecutando consulta: SELECT * FROM categories


,CategoryID,CategoryName
0,1,Confections
1,2,Shell fish
2,3,Cereals
3,4,Dairy
4,5,Beverages


In [4]:
df_customers = db.execute_query_as_dataframe("""
    SELECT c.CustomerID, c.FirstName, c.LastName, ct.CityName
    FROM customers c
    JOIN cities ct ON c.CityID = ct.CityID
""")
df_customers.head()

2025-06-08 06:15:10 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT c.CustomerID, c.FirstName, c.LastName, ct.CityName
    FROM customers c
    JOIN cities ct ON c.CityID = ct.CityID



,CustomerID,FirstName,LastName,CityName
0,1,Stefanie,Frye,Oklahoma
1,2,Sandy,Kirby,Pittsburgh
2,3,Lee,Zhang,Houston
3,4,Regina,Avery,Cleveland
4,5,Daniel,Mccann,Buffalo


In [5]:
df_products = db.execute_query_as_dataframe("""
    SELECT p.ProductID, p.ProductName, p.Price, cat.CategoryName
    FROM products p
    JOIN categories cat ON p.CategoryID = cat.CategoryID
""")
df_products.head()

2025-06-08 06:15:11 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT p.ProductID, p.ProductName, p.Price, cat.CategoryName
    FROM products p
    JOIN categories cat ON p.CategoryID = cat.CategoryID



,ProductID,ProductName,Price,CategoryName
0,1,Flour - Whole Wheat,74.2988,Cereals
1,2,Cookie Chocolate Chip With,91.2329,Cereals
2,3,Onions - Cippolini,9.1379,Poultry
3,4,Sauce - Gravy; Au Jus; Mix,54.3055,Poultry
4,5,Artichokes - Jerusalem,65.4771,Shell fish


In [6]:
df_sales = db.execute_query_as_dataframe("""
    SELECT SalesID, SalesDate, Quantity, TotalPrice, CustomerID
    FROM sales
    ORDER BY SalesDate DESC
    LIMIT 5
""")
df_sales.head()

2025-06-08 06:15:11 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT SalesID, SalesDate, Quantity, TotalPrice, CustomerID
    FROM sales
    ORDER BY SalesDate DESC
    LIMIT 5



,SalesID,SalesDate,Quantity,TotalPrice,CustomerID
0,417682,2 days 11:59:01,5,65.00,18075
1,3692623,2 days 11:59:01,3,51.00,8986
2,117578,2 days 11:59:00,5,45.00,17746
3,1944938,2 days 11:59:00,24,144.00,94027
4,2274474,2 days 11:59:00,16,80.00,61064


In [7]:
df_employees = db.execute_query_as_dataframe("""
    SELECT e.EmployeeID, e.FirstName, e.LastName, e.Gender, ct.CityName
    FROM employees e
    JOIN cities ct ON e.CityID = ct.CityID
""")
df_employees.head()

2025-06-08 06:15:11 - INFO - src.database.connection - Ejecutando consulta: 
    SELECT e.EmployeeID, e.FirstName, e.LastName, e.Gender, ct.CityName
    FROM employees e
    JOIN cities ct ON e.CityID = ct.CityID



,EmployeeID,FirstName,LastName,Gender,CityName
0,1,Nicole,Fuller,F,New Orleans
1,2,Christine,Palmer,F,Fremont
2,3,Pablo,Cline,M,Rochester
3,4,Darnell,Nielsen,M,Lubbock
4,5,Desiree,Stuart,F,Anaheim


### Patrón de diseño aplicado: Singleton 
La clase `DatabaseConnection` implementa el patrón Singleton, garantizando una única instancia global:
```python
db1 = DatabaseConnection()
db2 = DatabaseConnection()
assert db1 is db2
```

In [8]:
db1 = DatabaseConnection()
db2 = DatabaseConnection()
print(db1 is db2)

True


# AVANCE 3

## Creacion de estrategias de analisis de ventas - Patrón de diseño aplicado: Strategy + Facade 

Patron de diseño Facade, permite simplificar la interfaz para ejecutar multiples estrategias y ocultar la complejidad

In [49]:
from src.services.report_service import ReportService
report = ReportService(db)

En la carpeta sql/ se encuentran los scripts create_indexes.sql, create_procedures.sql y create_views.sql.

Estos scripts pueden ejecutarse directamente desde MySQL Workbench.

El método db.execute_query fue diseñado para la ejecución de queries simples sin retorno de DataFrames.

Nota:
El método db.execute_query es adecuado para sentencias como la creación de índices, procedimientos y vistas, siempre que el script no contenga instrucciones de cambio de delimitador (DELIMITER) ni múltiples comandos en un solo string. Para scripts complejos, se recomienda usar el cliente de MySQL o Workbench.

## Creacion de queries usando CTE + Funciones ventana y Objetos SQL

In [87]:
# Creamos una nueva vista utilizando

# Vista: vw_top_product_by_branch_hour_minute
# Descripción: Devuelve, para cada sucursal (ciudad), hora y minuto,
#              el producto más vendido (mayor cantidad) y su descuento.
# Uso:
#    SELECT * FROM vw_top_product_by_branch_hour_minute;

db.excute_query("""CREATE OR REPLACE VIEW vw_top_product_by_branch_hour_minute AS
WITH ventas_agrupadas AS (
    SELECT 
        ct.CityName AS sucursal,
        HOUR(s.SalesDate) AS hora,
        MINUTE(s.SalesDate) AS minuto,
        p.ProductName AS producto,
        s.Discount AS descuento,
        SUM(s.Quantity) AS cantidad_vendida,
        SUM(s.TotalPrice) AS total_ventas
    FROM sales s
    JOIN customers c ON s.CustomerID = c.CustomerID
    JOIN cities ct ON c.CityID = ct.CityID
    JOIN products p ON s.ProductID = p.ProductID
    GROUP BY sucursal, hora, minuto, producto, descuento
)
SELECT 
    sucursal,
    hora,
    minuto,
    producto,
    descuento,
    cantidad_vendida,
    total_ventas
FROM (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY sucursal, hora, minuto ORDER BY cantidad_vendida DESC) AS rn
    FROM ventas_agrupadas
) ranked
WHERE rn = 1
ORDER BY sucursal, hora, minuto;""")

2025-06-08 06:57:42 - INFO - src.database.connection - Ejecutando consulta: CREATE OR REPLACE VIEW vw_top_product_by_branch_hour_minute AS
WITH ventas_agrupadas AS (
    SELECT 
        ct.CityName AS sucursal,
        HOUR(s.SalesDate) AS hora,
        MINUTE(s.SalesDate) AS minuto,
        p.ProductName AS producto,
        s.Discount AS descuento,
        SUM(s.Quantity) AS cantidad_vendida,
        SUM(s.TotalPrice) AS total_ventas
    FROM sales s
    JOIN customers c ON s.CustomerID = c.CustomerID
    JOIN cities ct ON c.CityID = ct.CityID
    JOIN products p ON s.ProductID = p.ProductID
    GROUP BY sucursal, hora, minuto, producto, descuento
)
SELECT 
    sucursal,
    hora,
    minuto,
    producto,
    descuento,
    cantidad_vendida,
    total_ventas
FROM (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY sucursal, hora, minuto ORDER BY cantidad_vendida DESC) AS rn
    FROM ventas_agrupadas
) ranked
WHERE rn = 1
ORDER BY sucursal, hora, minuto;


Se utiliza el patrón Strategy para el manejo de múltiples estrategias con el fin de hacer la aplicación flexible, escalable y fácilmente extensible ante nuevos requerimientos de análisis, permitiendo agregar o modificar algoritmos de procesamiento sin afectar el resto del sistema.

In [ ]:
# Estrategia 1: sales_by_branch_and_hour para uso de la vista creada.

# Estrategia de análisis que identifica, para cada sucursal, hora y minuto,el producto más vendido y el descuento aplicado. Utiliza una consulta SQL con CTE y función ventana (ROW_NUMBER)

report.sales_by_branch_and_hour()

2025-06-08 06:58:05 - INFO - src.database.connection - Ejecutando consulta: SELECT * FROM vw_top_product_by_branch_hour_minute;


Tiempo de ejecución: 0.979 segundos
🔹 Ventas consolidadas por sucursal, hora/minuto y producto mas vendido


,sucursal,hora,minuto,producto,descuento,cantidad_vendida,total_ventas
0,Akron,0,0,Cheese - Cambozola,0.00,25,400.00
1,Akron,0,1,Ecolab - Mikroklene 4/4 L,0.00,1,5.00
2,Akron,0,2,Kellogs Special K Cereal,0.00,16,128.00
3,Akron,0,3,Ice Cream Bar - Hageen Daz To,0.00,18,216.00
4,Akron,0,5,Dc - Frozen Momji,0.10,11,143.00
...,...,...,...,...,...,...,...
46144,Yonkers,59,36,Soup - Campbells; Lentil,0.00,5,100.00
46145,Yonkers,59,41,Durian Fruit,0.00,12,144.00
46146,Yonkers,59,43,Juice - Orange,0.00,15,120.00
46147,Yonkers,59,47,Whmis - Spray Bottle Trigger,0.00,16,48.00


In [41]:
# Es posible dropear la vista, si se quieren realizar pruebas adicionales o de tolerancia a fallos

db.excute_query("""DROP VIEW vw_top_product_by_branch_hour_minute""")

2025-06-08 06:29:13 - INFO - src.database.connection - Ejecutando consulta: DROP VIEW vw_top_product_by_branch_hour_minute


In [52]:
# Estrategia 2: customer_behavior. Estrategia de análisis de patrones de comportamiento de los clientes.

#    Esta estrategia utiliza CTE y funciones ventana (ROW_NUMBER) para obtener:
#    - El total gastado, cantidad de transacciones y desglose de gasto con/sin promoción para cada cliente.
#    - La sucursal donde el cliente gastó más (ciudad de mayor gasto).
#    - El producto más comprado por cada cliente. 

report.customer_behavior()

2025-06-08 06:36:19 - INFO - src.database.connection - Ejecutando consulta: 
        WITH
          -- Gasto por cliente y sucursal
          gasto_por_sucursal AS (
              SELECT 
                  c.CustomerID,
                  ct.CityName AS sucursal,
                  SUM(s.TotalPrice) AS total_gastado_sucursal,
                  ROW_NUMBER() OVER (PARTITION BY c.CustomerID ORDER BY SUM(s.TotalPrice) DESC) AS rn_sucursal
              FROM customers c
              JOIN sales s ON c.CustomerID = s.CustomerID
              JOIN cities ct ON c.CityID = ct.CityID
              GROUP BY c.CustomerID, ct.CityName
          ),

          -- Cantidad comprada por cliente y producto
          productos_cliente AS (
              SELECT 
                  c.CustomerID,
                  p.ProductName,
                  SUM(s.Quantity) AS cantidad_comprada,
                  ROW_NUMBER() OVER (PARTITION BY c.CustomerID ORDER BY SUM(s.Quantity) DESC) AS rn_producto
              FROM 

Tiempo de ejecución: 0.986 segundos
🔹 Top clientes por gasto y aprovechamiento de promociones:


,CustomerID,cliente,total_gastado,transacciones,gastado_con_promocion,gastado_sin_promocion,sucursal_donde_mas_gasto,producto_mas_comprado
0,91038,Darcy Bullock,1872.00,4,0.00,1872.00,Oklahoma,Yogurt - Blueberry; 175 Gr
1,94115,Blake Dalton,1848.00,5,264.00,1584.00,Fremont,Tia Maria
2,96485,Forrest Morton,1700.00,3,0.00,1700.00,St. Paul,Sun - Dried Tomatoes
3,98273,Curtis Harmon,1650.00,4,825.00,825.00,St. Petersburg,Olives - Kalamata
4,73360,Allison Davies,1615.00,5,171.00,1444.00,Madison,Liners - Baking Cups
5,79369,John Gross,1491.00,4,0.00,1491.00,Columbus,Coffee - Irish Cream
6,97705,Fred Roberts,1475.00,3,900.00,575.00,Newark,Pork - Inside
7,95885,Aimee Banks,1475.00,4,625.00,850.00,Austin,Cookies Cereal Nut
8,94696,Spencer Booker,1440.00,5,408.00,1032.00,Cincinnati,Nut - Pistachio; Shelled
9,89007,Jolene Vincent,1426.00,4,437.00,989.00,Dayton,Tea - Earl Grey


In [56]:
# Creamos un nuevo Store procedure 

db.excute_query(""" 
CREATE PROCEDURE sp_top_products_by_branch(IN top_n INT)
BEGIN
    WITH productos_por_sucursal AS (
        SELECT
            ct.CityName AS sucursal,
            p.ProductID,
            p.ProductName,
            SUM(s.Quantity) AS unidades_vendidas,
            SUM(s.TotalPrice) AS total_facturado,
            SUM(CASE WHEN s.Discount > 0 THEN s.Quantity ELSE 0 END) AS unidades_con_promocion,
            SUM(CASE WHEN s.Discount = 0 THEN s.Quantity ELSE 0 END) AS unidades_sin_promocion,
            RANK() OVER (PARTITION BY ct.CityName ORDER BY SUM(s.Quantity) DESC) AS ranking
        FROM sales s
        JOIN products p ON s.ProductID = p.ProductID
        JOIN customers c ON s.CustomerID = c.CustomerID
        JOIN cities ct ON c.CityID = ct.CityID
        GROUP BY ct.CityName, p.ProductID, p.ProductName
    )
    SELECT *
    FROM productos_por_sucursal
    WHERE ranking <= top_n;
END
""")

2025-06-08 06:36:56 - INFO - src.database.connection - Ejecutando consulta:  
CREATE PROCEDURE sp_top_products_by_branch(IN top_n INT)
BEGIN
    WITH productos_por_sucursal AS (
        SELECT
            ct.CityName AS sucursal,
            p.ProductID,
            p.ProductName,
            SUM(s.Quantity) AS unidades_vendidas,
            SUM(s.TotalPrice) AS total_facturado,
            SUM(CASE WHEN s.Discount > 0 THEN s.Quantity ELSE 0 END) AS unidades_con_promocion,
            SUM(CASE WHEN s.Discount = 0 THEN s.Quantity ELSE 0 END) AS unidades_sin_promocion,
            RANK() OVER (PARTITION BY ct.CityName ORDER BY SUM(s.Quantity) DESC) AS ranking
        FROM sales s
        JOIN products p ON s.ProductID = p.ProductID
        JOIN customers c ON s.CustomerID = c.CustomerID
        JOIN cities ct ON c.CityID = ct.CityID
        GROUP BY ct.CityName, p.ProductID, p.ProductName
    )
    SELECT *
    FROM productos_por_sucursal
    WHERE ranking <= top_n;
END



In [ ]:
# Estrategia 3: product_performance utiliza el procedimiento almacenado. 

# Estrategia de análisis de rendimiento de productos por sucursal. Utiliza CTE y función ventana RANK para obtener, por cada sucursal (ciudad), el top 5 productos más vendidos, con detalle de unidades vendidas, facturación total y ventas con/sin promoción.

report.product_performance() # Por defecto se toma el top de 5 productos mas vendidos

2025-06-08 07:00:15 - INFO - src.database.connection - Ejecutando consulta: CALL sp_top_products_by_branch(5);


Tiempo de ejecución: 0.482 segundos
🔹 Top 5 productos más vendidos por sucursal:


,sucursal,ProductID,ProductName,unidades_vendidas,total_facturado,unidades_con_promocion,unidades_sin_promocion,ranking
0,Akron,14,Beef - Top Sirloin,70,1106.00,23,47,1
1,Akron,416,Baking Powder,68,1236.00,24,44,2
2,Akron,404,Pants Custom Dry Clean,68,705.00,23,45,2
3,Akron,396,Tea - Jasmin Green,66,698.00,50,16,4
4,Akron,314,Salmon Steak - Cohoe 8 Oz,64,919.00,37,27,5
...,...,...,...,...,...,...,...,...
500,Yonkers,14,Beef - Top Sirloin,80,808.00,36,44,1
501,Yonkers,81,Cookies - Assorted,73,668.00,33,40,2
502,Yonkers,27,Chocolate - Compound Coating,72,967.00,15,57,3
503,Yonkers,375,Snapple Lemon Tea,70,461.00,0,70,4


In [91]:
# Es posible modificar el parametro de entrada para elegir el RANKING del top de productos mas vendidos

report.product_performance(2)

2025-06-08 07:00:35 - INFO - src.database.connection - Ejecutando consulta: CALL sp_top_products_by_branch(2);


Tiempo de ejecución: 0.512 segundos
🔹 Top 2 productos más vendidos por sucursal:


,sucursal,ProductID,ProductName,unidades_vendidas,total_facturado,unidades_con_promocion,unidades_sin_promocion,ranking
0,Akron,14,Beef - Top Sirloin,70,1106.00,23,47,1
1,Akron,416,Baking Powder,68,1236.00,24,44,2
2,Akron,404,Pants Custom Dry Clean,68,705.00,23,45,2
3,Albuquerque,186,Muffin Mix - Blueberry,83,1265.00,3,80,1
4,Albuquerque,250,Soup - Campbells; Beef Barley,72,510.00,0,72,2
...,...,...,...,...,...,...,...,...
196,Washington,54,Liners - Banana; Paper,76,802.00,48,28,2
197,Wichita,196,Longos - Grilled Salmon With Bbq,108,1798.00,19,89,1
198,Wichita,93,Bandage - Fexible 1x3,84,1162.00,15,69,2
199,Yonkers,14,Beef - Top Sirloin,80,808.00,36,44,1


In [54]:
# Es posible dropear el procedimiento almacenado, si se quieren realizar pruebas adicionales o de tolerancia a fallos

db.excute_query("""DROP PROCEDURE sp_top_products_by_branch;""")

2025-06-08 06:36:48 - INFO - src.database.connection - Ejecutando consulta: DROP PROCEDURE sp_top_products_by_branch;


## Implementacion de indices

En este entorno y con la cantidad de datos actual, la creación de índices no tuvo impacto visible en los tiempos de ejecución del store procedure ( y en general ). Esto se debe a que el volumen de datos es moderado y la consulta realiza agregaciones sobre toda la tabla, donde el optimizador de MySQL prefiere un escaneo completo.

En escenarios productivos con millones de registros y consultas filtradas, los índices aportan mejoras significativas en la performance.

In [69]:
# Verificacion de consulta sin el uso de indices
db.execute_query_as_dataframe("""EXPLAIN SELECT * FROM vw_top_product_by_branch_hour_minute;""")

2025-06-08 06:47:23 - INFO - src.database.connection - Ejecutando consulta: EXPLAIN SELECT * FROM vw_top_product_by_branch_hour_minute;


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,<derived3>,None,ref,<auto_key0>,<auto_key0>,8,const,10,100.0,Using filesort
1,3,DERIVED,<derived4>,None,ALL,None,None,None,None,49859,100.0,Using filesort
2,4,DERIVED,s,None,ALL,None,None,None,None,49859,100.0,Using where; Using temporary
3,4,DERIVED,c,None,eq_ref,PRIMARY,PRIMARY,4,sales_company.s.CustomerID,1,100.0,Using where
4,4,DERIVED,ct,None,eq_ref,PRIMARY,PRIMARY,4,sales_company.c.CityID,1,100.0,None
5,4,DERIVED,p,None,eq_ref,PRIMARY,PRIMARY,4,sales_company.s.ProductID,1,100.0,None


In [ ]:
#Índices recomendados para optimizacion en las consultas

# Acelera JOINs y agrupamientos por cliente en ventas
db.excute_query("""CREATE INDEX idx_sales_customerid ON sales(CustomerID);""")

# Acelera JOINs y agrupamientos por producto en ventas
db.excute_query("""CREATE INDEX idx_sales_productid  ON sales(ProductID) ;""")

# Acelera búsquedas y agrupaciones por fecha/hora de venta
db.excute_query("""CREATE INDEX idx_sales_salesdate  ON sales(SalesDate) ;""")

2025-06-08 06:51:31 - INFO - src.database.connection - Ejecutando consulta: CREATE INDEX idx_sales_customerid ON sales(CustomerID);
2025-06-08 06:51:31 - INFO - src.database.connection - Ejecutando consulta: CREATE INDEX idx_sales_productid  ON sales(ProductID) ;
2025-06-08 06:51:31 - INFO - src.database.connection - Ejecutando consulta: CREATE INDEX idx_sales_salesdate  ON sales(SalesDate) ;


Los tiempos de ejecución de las diferentes estrategias pueden incluso empeorar después de la incorporación de índices, debido a que en consultas de agregación o agrupamiento sobre grandes volúmenes de datos, el motor de MySQL puede preferir realizar un escaneo completo de la tabla en vez de utilizar los índices. Además, la presencia de índices puede introducir una sobrecarga en operaciones de escritura o mantenimiento, y en algunos casos el optimizador puede elegir un plan de ejecución subóptimo, resultando en un desempeño inferior al esperado.

In [86]:
# Verificacion del uso de indices en las consultas
db.execute_query_as_dataframe("""EXPLAIN SELECT * FROM vw_top_product_by_branch_hour_minute;""")

2025-06-08 06:51:49 - INFO - src.database.connection - Ejecutando consulta: EXPLAIN SELECT * FROM vw_top_product_by_branch_hour_minute;


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,PRIMARY,<derived3>,None,ref,<auto_key0>,<auto_key0>,8,const,10,100.0,Using filesort
1,3,DERIVED,<derived4>,None,ALL,None,None,None,None,49421,100.0,Using filesort
2,4,DERIVED,p,None,ALL,PRIMARY,None,None,None,452,100.0,Using temporary
3,4,DERIVED,s,None,ref,"idx_sales_customerid,idx_sales_productid",idx_sales_productid,5,sales_company.p.ProductID,109,100.0,Using where
4,4,DERIVED,c,None,eq_ref,PRIMARY,PRIMARY,4,sales_company.s.CustomerID,1,100.0,Using where
5,4,DERIVED,ct,None,eq_ref,PRIMARY,PRIMARY,4,sales_company.c.CityID,1,100.0,None


In [84]:
report.sales_by_branch_and_hour()

2025-06-08 06:51:37 - INFO - src.database.connection - Ejecutando consulta: SELECT * FROM vw_top_product_by_branch_hour_minute;


Tiempo de ejecución: 1.211 segundos
🔹 Ventas consolidadas por sucursal, hora/minuto y producto mas vendido


,sucursal,hora,minuto,producto,descuento,cantidad_vendida,total_ventas
0,Akron,0,0,Cheese - Cambozola,0.00,25,400.00
1,Akron,0,1,Ecolab - Mikroklene 4/4 L,0.00,1,5.00
2,Akron,0,2,Kellogs Special K Cereal,0.00,16,128.00
3,Akron,0,3,Ice Cream Bar - Hageen Daz To,0.00,18,216.00
4,Akron,0,5,Dc - Frozen Momji,0.10,11,143.00
...,...,...,...,...,...,...,...
46144,Yonkers,59,36,Soup - Campbells; Lentil,0.00,5,100.00
46145,Yonkers,59,41,Durian Fruit,0.00,12,144.00
46146,Yonkers,59,43,Juice - Orange,0.00,15,120.00
46147,Yonkers,59,47,Whmis - Spray Bottle Trigger,0.00,16,48.00


In [77]:
report.customer_behavior()

2025-06-08 06:51:16 - INFO - src.database.connection - Ejecutando consulta: 
        WITH
          -- Gasto por cliente y sucursal
          gasto_por_sucursal AS (
              SELECT 
                  c.CustomerID,
                  ct.CityName AS sucursal,
                  SUM(s.TotalPrice) AS total_gastado_sucursal,
                  ROW_NUMBER() OVER (PARTITION BY c.CustomerID ORDER BY SUM(s.TotalPrice) DESC) AS rn_sucursal
              FROM customers c
              JOIN sales s ON c.CustomerID = s.CustomerID
              JOIN cities ct ON c.CityID = ct.CityID
              GROUP BY c.CustomerID, ct.CityName
          ),

          -- Cantidad comprada por cliente y producto
          productos_cliente AS (
              SELECT 
                  c.CustomerID,
                  p.ProductName,
                  SUM(s.Quantity) AS cantidad_comprada,
                  ROW_NUMBER() OVER (PARTITION BY c.CustomerID ORDER BY SUM(s.Quantity) DESC) AS rn_producto
              FROM 

Tiempo de ejecución: 0.948 segundos
🔹 Top clientes por gasto y aprovechamiento de promociones:


,CustomerID,cliente,total_gastado,transacciones,gastado_con_promocion,gastado_sin_promocion,sucursal_donde_mas_gasto,producto_mas_comprado
0,91038,Darcy Bullock,1872.00,4,0.00,1872.00,Oklahoma,Yogurt - Blueberry; 175 Gr
1,94115,Blake Dalton,1848.00,5,264.00,1584.00,Fremont,Tia Maria
2,96485,Forrest Morton,1700.00,3,0.00,1700.00,St. Paul,Sun - Dried Tomatoes
3,98273,Curtis Harmon,1650.00,4,825.00,825.00,St. Petersburg,Olives - Kalamata
4,73360,Allison Davies,1615.00,5,171.00,1444.00,Madison,Liners - Baking Cups
5,79369,John Gross,1491.00,4,0.00,1491.00,Columbus,Coffee - Irish Cream
6,97705,Fred Roberts,1475.00,3,900.00,575.00,Newark,Pork - Inside
7,95885,Aimee Banks,1475.00,4,625.00,850.00,Austin,Cookies Cereal Nut
8,94696,Spencer Booker,1440.00,5,408.00,1032.00,Cincinnati,Nut - Pistachio; Shelled
9,89007,Jolene Vincent,1426.00,4,437.00,989.00,Dayton,Tea - Earl Grey


In [85]:
report.product_performance()

2025-06-08 06:51:42 - INFO - src.database.connection - Ejecutando consulta: CALL sp_top_products_by_branch(5);


Tiempo de ejecución: 0.468 segundos
🔹 Top 5 productos más vendidos por sucursal:


,sucursal,ProductID,ProductName,unidades_vendidas,total_facturado,unidades_con_promocion,unidades_sin_promocion,ranking
0,Akron,14,Beef - Top Sirloin,70,1106.00,23,47,1
1,Akron,416,Baking Powder,68,1236.00,24,44,2
2,Akron,404,Pants Custom Dry Clean,68,705.00,23,45,2
3,Akron,396,Tea - Jasmin Green,66,698.00,50,16,4
4,Akron,314,Salmon Steak - Cohoe 8 Oz,64,919.00,37,27,5
...,...,...,...,...,...,...,...,...
500,Yonkers,14,Beef - Top Sirloin,80,808.00,36,44,1
501,Yonkers,81,Cookies - Assorted,73,668.00,33,40,2
502,Yonkers,27,Chocolate - Compound Coating,72,967.00,15,57,3
503,Yonkers,375,Snapple Lemon Tea,70,461.00,0,70,4


In [75]:
# Es posible eliminar los indices, para realizar nuevas pruebas en los tiempos de ejecucion. 

db.excute_query("""DROP INDEX idx_sales_customerid ON sales;""")
db.excute_query("""DROP INDEX idx_sales_productid ON sales;""")
db.excute_query("""DROP INDEX idx_sales_salesdate ON sales;""")


2025-06-08 06:51:08 - INFO - src.database.connection - Ejecutando consulta: DROP INDEX idx_sales_customerid ON sales;
2025-06-08 06:51:08 - INFO - src.database.connection - Ejecutando consulta: DROP INDEX idx_sales_productid ON sales;
2025-06-08 06:51:08 - INFO - src.database.connection - Ejecutando consulta: DROP INDEX idx_sales_salesdate ON sales;


# Ejecucion de test

In [92]:
#Ejecución de pruebas unitarias incluyendo el test unitario de una unica conexion a la base de datos
!pytest ../test -v

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.0.2, pluggy-1.5.0 -- C:\Users\bcami\OneDrive\Escritorio\ACCENTURE\sales_company\venv_sales_company\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\bcami\OneDrive\Escritorio\ACCENTURE\sales_company
plugins: anyio-4.8.0, cov-6.0.0
collecting ... collected 12 items

..\test\test_customer.py::test_get_full_name PASSED                      [  8%]
..\test\test_customer.py::test_set_address PASSED                        [ 16%]
..\test\test_customer.py::test_customer_repr PASSED                      [ 25%]
..\test\test_database_connection.py::test_singleton_instance PASSED      [ 33%]
..\test\test_database_connection.py::test_get_session PASSED             [ 41%]
..\test\test_database_connection.py::test_close_session PASSED           [ 50%]
..\test\test_database_connection.py::test_execute_query_as_dataframe PASSED [ 58%]
..\test\test_employee.py::test_employe